<span style="color:red">**IMPORTANT**: If updates are made to the `filtered` subsets data, it's important to merge them to the `items` files using `merge_subset.ipynb` before running this file or any updates made will be wiped</span>

In [41]:
from utils.utils import load_json, save_json

In [42]:
"""Loads OSRSbox weapon datasets and appends the missing weapons JSON files (which contain new weapons introduced after ~2020).
"""


def load_weapons():
    weapons = load_json("data/items-weapon.json")
    missing_data = load_json("data/missing_data.json")

    for weapon_id, weapon_info in missing_data.items():
        weapons[str(weapon_id)] = weapon_info

    return weapons


def load_2h_weapons():
    two_hand_weapons = load_json("data/items-2h.json")
    missing_2h_data = load_json("data/missing_2h_data.json")

    for weapon_id, weapon_info in missing_2h_data.items():
        two_hand_weapons[str(weapon_id)] = weapon_info

    return two_hand_weapons


weapons = load_weapons()
two_hand_weapons = load_2h_weapons()

In [43]:
"""The code in this cell as reduces the OSRS weapons data down to standard mid to late game PVE melee weapons. Trivial copies like poisoned variants or minigame variants were removed. Weapons that are not feasible or realistic for the average player or are more decorative, like 3rd age weapons, volcanic/frozen whips, ornate or gilded weapons were removed. Weapons used primarily for skilling were removed. Low durability or partially charged variants like those for barrows or crystal weapons were removed.

Additionally, it is a very old game, and select OSRS weapons have multiple identical copies or duplicates under different item ids. The duplicates item ids have been added to lists to be removed. 
"""

filter_weapons_substring = [
    "last man standing", "(or)", "(c)", "(poison)", "(poison+)",
    "(poison++)", "(p)", "(p+)", "(p++)", "(attuned)",
    "(basic)", "(perfected)", "corrupted halberd",
    "salamander", "adamant", "(75)", "(50)", "(25)", "(inactive)",
    "(historical)", "(active)", "dinh's", "(100)",
    "(kp)", "gilded", "(ornate handle)", "mithril",
    "(uncharged)", "pharaoh's", "holy", "sanguine", "(partially charged)",
    "(t)", "3rd", "staff", "battlestaff", "crozier", "barronite",
    "decorative", "(karambwan poison)", "cane", "cleaver", "sickle",
    "pickaxe", "harpoon", "infernal axe", "frozen", "meat", "rolling",
    "skewer", "vesta's", "statius's", "trailblazer axe", "rune",
    "dragon axe", "void", "volcanic", "(i)", "brine", "gadderhammer",
    "hill giant", "toktz-mej-tal", "blisterwood", "ivandis",
]
weapon_duplicates = ["1373", "7668", "21205", "20559", "10581", "7142"]
two_hand_weapon_duplicates = ["20402", "23332", "23330", "1333", "21009", "20785", "7142", "10581", "4587", "6523"]
keep_these_key_values = ["id", "name", "wiki_name", "wiki_url", "equipment", "weapon"]


def filter_weapons(weapons, duplicates, filter_weapons_substring):
    filtered_weapons = {}

    for weapon_id, weapon_info in weapons.items():
        if (weapon_info["equipment"]["melee_strength"] >= 30
                and weapon_id not in duplicates
                and not any(substring in weapon_info["wiki_name"].lower() for substring in filter_weapons_substring)):
            weapon_info_copy = {}
            for key, value in weapon_info.items():
                if key in keep_these_key_values:
                    weapon_info_copy[key] = value
            filtered_weapons[weapon_id] = weapon_info_copy

    return dict(sorted(filtered_weapons.items(), key=lambda weapon: weapon[1]['wiki_name']))


strong_weapons = filter_weapons(weapons, weapon_duplicates, filter_weapons_substring)
strong_two_hand_weapons = filter_weapons(two_hand_weapons, two_hand_weapon_duplicates, filter_weapons_substring)

save_json("data/filtered_weapons.json", strong_weapons)
save_json("data/filtered_2h_weapons.json", strong_two_hand_weapons)

In [44]:
"""This cell retrieves select non-spec weapons that are considered meta for OSRS using their item ids and saves them to a JSON file. It mostly retrieves the top three most common and strongest crush, slash, and stab weapons with a few additional weapons like the new Soulreaper Axe. As stated, meta "spec" or special attack weapons like Elder Maul, Dragon Claws and Voidwaker are omitted.
"""

meta_weapon_id = ["4151", "12006", "23995", "20406", "22324", "24417", "25977", "12346", "23528", "11889"]
meta_two_hand_weapon_id = ["13263", "12355", "22325", "12357"]


def filter_meta_weapons(weapons, meta_weapon_id):
    filtered_meta_weapons = {}

    for weapon_id, weapon_info in weapons.items():
        if weapon_id in meta_weapon_id:
            weapon_info_copy = {}
            for key, value in weapon_info.items():
                if key in keep_these_key_values:
                    weapon_info_copy[key] = value
            filtered_meta_weapons[weapon_id] = weapon_info_copy

    return dict(sorted(filtered_meta_weapons.items(), key=lambda weapon: weapon[1]['wiki_name']))


meta_weapons = filter_meta_weapons(weapons, meta_weapon_id)
meta_two_hand_weapons = filter_meta_weapons(two_hand_weapons, meta_two_hand_weapon_id)

save_json("data/filtered_meta_weapons.json", meta_weapons)
save_json("data/filtered_meta_two_hand_weapons.json", meta_two_hand_weapons)